In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.stats as stats
mpl.rc('font', family='Malgun Gothic')

In [2]:
df_member = pd.read_csv('1_store_member.csv')
df_product = pd.read_csv('2_store_product.csv')

In [3]:
df_member.head(2)

,회원번호,회원상태,성별,결혼유무,주소,생년,연령
0,2101,정상회원,여,기혼,서울 강북구 미아동 134-,1967,51
1,2102,정상회원,여,NaN,경기 용인시 수지구 동천동,1947,71


In [4]:
df_product.head(2)

,공급일자,물품코드,물품대분류,물품중분류,물품소분류,물품명,구매수량,구매금액,구매매장,반품_원거래일자,회원번호
0,20170201,50142001,채소,잎/줄기채소,시금치,시금치(300g),1.0,2000,매장3,NaN,2101.0
1,20170201,60302012,축산물,육가공,돈가스/분쇄가공,튀김옷돈가스(400g),1.0,6300,매장3,NaN,2101.0


In [5]:
# 구매 데이터에서 회원 번호 별 총 구매금액 / 총구매수량 계산
p1 = df_product.pivot_table(index='회원번호', 
                            values=['구매금액', '구매수량'], 
                            aggfunc='sum').reset_index()

In [6]:
df_member2 = pd.merge(df_member, p1, on='회원번호', how='inner')

In [7]:
# 구매 데이터에서 회원 번호 별 방문횟수의 총합
df_product['방문횟수'] = 1

In [8]:
p2 = df_product.pivot_table(index=['회원번호', '공급일자'], 
                            values='방문횟수', aggfunc='min').reset_index()
p3 = p2.pivot_table(index='회원번호', values = '방문횟수', 
                    aggfunc='sum').reset_index()

In [9]:
df_member3 = pd.merge(df_member2, p3, on='회원번호', how='inner')

In [10]:
df_member3['1회방문구매평균'] = df_member3['구매금액'] / df_member3['방문횟수']
df_member3['1회방문구매평균'].describe()

count      4285.000000
mean      32913.468270
std       25756.734483
min           0.000000
25%       15890.476190
50%       27166.666667
75%       43080.140845
max      329260.000000
Name: 1회방문구매평균, dtype: float64

In [11]:
# 1회방문구매평균을 바탕으로 고객등급을 계산 (연속형 -> 범주형, 층별화)
def func1(row):
    if row > 100000:
        return 'A'
    elif row>=50000:
        return 'B'
    elif row>=20000:
        return 'C'
    else:
        return 'D'

df_member3['고객등급'] = df_member3['1회방문구매평균'].apply(func1)

In [12]:
df_member3['고객등급'].value_counts()

C    2006
D    1506
B     676
A      97
Name: 고객등급, dtype: int64

In [13]:
# 데이터 재구조화
df1 = pd.read_excel('04_Data.xlsx',skiprows=1) # 엑셀에서의 첫번째 줄
df1.shape

(40, 25)

In [14]:
df1.columns

Index([                   '구분',                 '카테고리명',
                        '자재그룹',                 '자재그룹명',
                         '제품명',                    '단위',
                          '판매',     2019-12-01 00:00:00,
           2020-10-01 00:00:00,     2020-11-01 00:00:00,
           2020-12-01 00:00:00, '2019-12-01 00:00:00.1',
       '2020-10-01 00:00:00.1', '2020-11-01 00:00:00.1',
       '2020-12-01 00:00:00.1', '2019-12-01 00:00:00.2',
       '2020-10-01 00:00:00.2', '2020-11-01 00:00:00.2',
       '2020-12-01 00:00:00.2', '2019-12-01 00:00:00.3',
       '2020-10-01 00:00:00.3', '2020-11-01 00:00:00.3',
       '2020-12-01 00:00:00.3',                  '안전재고',
                         ' 분류'],
      dtype='object')

In [15]:
#재구조화 Reshape
df2 = df1.melt(id_vars=['구분', '카테고리명', '자재그룹', '자재그룹명', 
                       '제품명', '단위', '판매', '안전재고', ' 분류'])   # 분류 앞에 띄어쓰기 주의
df2

,구분,카테고리명,자재그룹,자재그룹명,제품명,단위,판매,안전재고,분류,variable,value
0,구분,CHB,1687,친환경 바디워시,A,천원,0.0,0.0,E,2019-12-01 00:00:00.000,0.0
1,구분,CHB,1687,친환경 바디워시,B,천원,0.0,0.0,E,2019-12-01 00:00:00.000,0.0
2,구분,CHB,1687,친환경 바디워시,C,천원,10354.1,0.0,A,2019-12-01 00:00:00.000,9474.3
3,구분,CHB,1687,친환경 바디워시,D,천원,3543.3,0.0,A,2019-12-01 00:00:00.000,7702.6
4,구분,CHB,1687,친환경 바디워시,E,천원,234603.9,39593.3,A,2019-12-01 00:00:00.000,56309.6
...,...,...,...,...,...,...,...,...,...,...,...
635,구분,CHB,1687,친환경 바디워시,Z2,천원,5012.4,0.0,K,2020-12-01 00:00:00.300,1619.5
636,구분,CHB,1687,친환경 바디워시,Z3,천원,585.0,0.0,K,2020-12-01 00:00:00.300,3191.2
637,구분,CHB,1687,친환경 바디워시,KC,천원,0.0,0.0,E,2020-12-01 00:00:00.300,0.0
638,구분,CHB,1687,친환경 바디워시,KC2,천원,16636.2,0.0,L,2020-12-01 00:00:00.300,16845.8


In [16]:
#항목명을 변경
df3 = df2.rename(columns={'variable':'날짜', 
                          'value': '판매금액', ' 분류':'분류'})
df3

,구분,카테고리명,자재그룹,자재그룹명,제품명,단위,판매,안전재고,분류,날짜,판매금액
0,구분,CHB,1687,친환경 바디워시,A,천원,0.0,0.0,E,2019-12-01 00:00:00.000,0.0
1,구분,CHB,1687,친환경 바디워시,B,천원,0.0,0.0,E,2019-12-01 00:00:00.000,0.0
2,구분,CHB,1687,친환경 바디워시,C,천원,10354.1,0.0,A,2019-12-01 00:00:00.000,9474.3
3,구분,CHB,1687,친환경 바디워시,D,천원,3543.3,0.0,A,2019-12-01 00:00:00.000,7702.6
4,구분,CHB,1687,친환경 바디워시,E,천원,234603.9,39593.3,A,2019-12-01 00:00:00.000,56309.6
...,...,...,...,...,...,...,...,...,...,...,...
635,구분,CHB,1687,친환경 바디워시,Z2,천원,5012.4,0.0,K,2020-12-01 00:00:00.300,1619.5
636,구분,CHB,1687,친환경 바디워시,Z3,천원,585.0,0.0,K,2020-12-01 00:00:00.300,3191.2
637,구분,CHB,1687,친환경 바디워시,KC,천원,0.0,0.0,E,2020-12-01 00:00:00.300,0.0
638,구분,CHB,1687,친환경 바디워시,KC2,천원,16636.2,0.0,L,2020-12-01 00:00:00.300,16845.8


In [17]:
df3.isnull().sum()

구분       0
카테고리명    0
자재그룹     0
자재그룹명    0
제품명      0
단위       0
판매       0
안전재고     0
분류       0
날짜       0
판매금액     0
dtype: int64

In [18]:
df3.shape

(640, 11)

In [19]:
df3['날짜']

# 날짜 자료를 엑셀에서 가져오면 csv와 다르게 날짜 데이터 타입으로 가져온다
# csv와 excel이 같은 개수만큼의 데이터가 있다면 정형구조에 대한 데이터를 갖는 excel의 용량이 더 크다
# excel : 정형구조 (index, column, value)

0     2019-12-01 00:00:00.000
1     2019-12-01 00:00:00.000
2     2019-12-01 00:00:00.000
3     2019-12-01 00:00:00.000
4     2019-12-01 00:00:00.000
                ...          
635   2020-12-01 00:00:00.300
636   2020-12-01 00:00:00.300
637   2020-12-01 00:00:00.300
638   2020-12-01 00:00:00.300
639   2020-12-01 00:00:00.300
Name: 날짜, Length: 640, dtype: datetime64[ns]

In [20]:
df3['공급연도'] = df3['날짜'].dt.year
df3['공급연도'].value_counts()

2020    480
2019    160
Name: 공급연도, dtype: int64

In [21]:
# 분류별 19년도 대비 20년도의 판매금액의 증감 확인
p1 = df3.pivot_table(index='분류', columns='공급연도', 
                     values='판매금액', aggfunc='sum').reset_index()
p1['증감률'] = (p1[2020] - p1[2019]) / p1[2019]

In [22]:
p1

공급연도,분류,2019,2020,증감률
0,A,356379.4,1448912.0,3.065645
1,B,20564.1,120.0,-0.994165
2,E,27600.5,25506.2,-0.075879
3,F,19498.5,113960.6,4.844583
4,G,1039.0,25810.2,23.841386
5,H,0.0,52900.4,inf
6,K,88588.6,105500.9,0.190908
7,L,0.0,107632.8,inf
8,M,143063.8,296561.9,1.072935
